In [4]:
import pandas as pd
import numpy as np

In [5]:
df_source = pd.read_csv('regression_mc_sim.csv')

In [22]:
df_source[df_source['LapNumber']==23]

,LapNumber,TrackTemp,RPM,Throttle,Brake,DRS,LapTime,IsPersonalBest,Compound,TyreLife,FreshTyre,TrackStatus,Weight,PitDuration
22,23.0,34.3,9566.787879,51.506061,81,0,78.431,1,1.0,27.0,0,0,809.061557,0.0
100,23.0,34.3,9537.425000,47.009375,97,0,78.373,1,3.0,2.0,1,0,808.073223,0.0
178,23.0,34.3,9216.135952,44.595166,99,0,77.018,1,3.0,4.0,1,0,807.905372,0.0
230,23.0,34.3,9454.107034,49.639144,99,18,78.184,0,3.0,9.0,1,0,807.815128,0.0
300,23.0,34.3,9234.946108,43.161677,110,0,83.528,0,3.0,15.0,0,0,809.418369,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7941,23.0,46.3,9128.176667,43.920000,75,16,80.662,1,6.0,23.0,0,0,876.685653,0.0
8016,23.0,46.3,8516.153333,41.173333,97,15,80.750,0,6.0,9.0,0,0,875.091548,0.0
8091,23.0,46.3,8915.910891,39.920792,78,0,81.188,0,5.0,23.0,0,0,872.698964,0.0
8166,23.0,46.3,9112.856667,40.130000,88,20,80.593,1,6.0,23.0,0,0,870.899698,0.0


In [11]:
laps = pd.DataFrame({'LapNumber': range(1, 79)})

# ---- Step 2: Aggregate fixed variables from source ----
agg_fixed = df_source.groupby('LapNumber').agg({
    'Throttle': 'mean',
    'Brake': 'mean',
    'DRS': 'mean'
}).reset_index()

# Round Throttle, Brake, DRS to integers
agg_fixed['Brake'] = agg_fixed['Brake'].round().astype(int)
agg_fixed['DRS'] = agg_fixed['DRS'].round().astype(int)

# Merge into the lap DataFrame
laps = laps.merge(agg_fixed, on='LapNumber', how='left')

# ---- Step 3: Pit Duration ----
# Average of nonzero pit_durations
avg_pit_duration = df_source[df_source['PitDuration'] > 0]['PitDuration'].mean()

# Assign pit_duration
laps['PitDuration'] = 0
laps.loc[laps['LapNumber'] == 32, 'PitDuration'] = avg_pit_duration

# ---- Step 4: Compound ----
laps['Compound'] = np.where(laps['LapNumber'] <= 31, 5, 6)

# ---- Step 5: Tyre Life ----
laps['TyreLife'] = np.where(
    laps['LapNumber'] <= 31,
    laps['LapNumber'],                   # Laps 1–31
    laps['LapNumber'] - 31                # Restart count from 1 after pit stop
)

C:\Users\prath\AppData\Local\Temp\ipykernel_31572\1575569248.py:23: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '25.96884722222222' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  laps.loc[laps['LapNumber'] == 32, 'PitDuration'] = avg_pit_duration


In [12]:
laps

,LapNumber,Throttle,Brake,DRS,PitDuration,Compound,TyreLife
0,1,40.101048,112,0,0.0,5,1
1,2,41.665370,102,5,0.0,5,2
2,3,41.956385,98,7,0.0,5,3
3,4,42.088928,97,7,0.0,5,4
4,5,42.470642,98,5,0.0,5,5
...,...,...,...,...,...,...,...
73,74,45.910959,91,4,0.0,6,43
74,75,44.908604,91,4,0.0,6,44
75,76,42.370703,104,3,0.0,6,45
76,77,36.147459,128,3,0.0,6,46
